In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd drive/MyDrive/FTSL/

/content/drive/.shortcut-targets-by-id/1CI1F_BNtclDb589EJiZx_ypJ_TSazyxZ/FTSL


In [ ]:
!pip3 install tensorflow_model_optimization

In [ ]:
import os
import sys
import tempfile

import cv2
import keras
import numpy as np
from numpy import dstack 
import pandas as pd
from keras.layers import (Conv2D, Dense, Dropout, Flatten, GaussianNoise,
                          MaxPooling2D, BatchNormalization)
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.models import Sequential
from keras.utils import np_utils, to_categorical
from matplotlib import pyplot
from numpy import dstack, mean, std
from pandas import read_csv
from PIL import Image
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

%matplotlib inline
import random

import matplotlib.pyplot as plt
import numpy
import plotly.express as px
import requests
import tensorflow as tf
import tensorflow_model_optimization as tfmot
from scipy.spatial.distance import euclidean as euc

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

from plotly.offline import download_plotlyjs, init_notebook_mode, iplot

init_notebook_mode(connected=True)

In [ ]:
DF_train = pd.read_csv('./train.csv')
DF_test = pd.read_csv('./test.csv')
DF = pd.concat([DF_train, DF_test], axis = 0)
DF = DF.sample(frac=1).reset_index(drop=True)
DF

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,tBodyAcc-max()-Y,tBodyAcc-max()-Z,tBodyAcc-min()-X,tBodyAcc-min()-Y,tBodyAcc-min()-Z,tBodyAcc-sma(),tBodyAcc-energy()-X,tBodyAcc-energy()-Y,tBodyAcc-energy()-Z,tBodyAcc-iqr()-X,tBodyAcc-iqr()-Y,tBodyAcc-iqr()-Z,tBodyAcc-entropy()-X,tBodyAcc-entropy()-Y,tBodyAcc-entropy()-Z,"tBodyAcc-arCoeff()-X,1","tBodyAcc-arCoeff()-X,2","tBodyAcc-arCoeff()-X,3","tBodyAcc-arCoeff()-X,4","tBodyAcc-arCoeff()-Y,1","tBodyAcc-arCoeff()-Y,2","tBodyAcc-arCoeff()-Y,3","tBodyAcc-arCoeff()-Y,4","tBodyAcc-arCoeff()-Z,1","tBodyAcc-arCoeff()-Z,2","tBodyAcc-arCoeff()-Z,3","tBodyAcc-arCoeff()-Z,4","tBodyAcc-correlation()-X,Y","tBodyAcc-correlation()-X,Z","tBodyAcc-correlation()-Y,Z",...,fBodyBodyAccJerkMag-entropy(),fBodyBodyAccJerkMag-maxInds,fBodyBodyAccJerkMag-meanFreq(),fBodyBodyAccJerkMag-skewness(),fBodyBodyAccJerkMag-kurtosis(),fBodyBodyGyroMag-mean(),fBodyBodyGyroMag-std(),fBodyBodyGyroMag-mad(),fBodyBodyGyroMag-max(),fBodyBodyGyroMag-min(),fBodyBodyGyroMag-sma(),fBodyBodyGyroMag-energy(),fBodyBodyGyroMag-iqr(),fBodyBodyGyroMag-entropy(),fBodyBodyGyroMag-maxInds,fBodyBodyGyroMag-meanFreq(),fBodyBodyGyroMag-skewness(),fBodyBodyGyroMag-kurtosis(),fBodyBodyGyroJerkMag-mean(),fBodyBodyGyroJerkMag-std(),fBodyBodyGyroJerkMag-mad(),fBodyBodyGyroJerkMag-max(),fBodyBodyGyroJerkMag-min(),fBodyBodyGyroJerkMag-sma(),fBodyBodyGyroJerkMag-energy(),fBodyBodyGyroJerkMag-iqr(),fBodyBodyGyroJerkMag-entropy(),fBodyBodyGyroJerkMag-maxInds,fBodyBodyGyroJerkMag-meanFreq(),fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",subject,Activity
0,0.275800,0.001837,-0.136288,-0.036102,0.284849,-0.141780,-0.164648,0.147255,-0.175408,0.396171,-0.027748,-0.300698,0.060519,-0.289720,0.007840,0.020844,-0.534534,-0.681412,-0.666986,-0.527540,-0.370368,-0.209740,0.028807,0.554978,0.206736,-0.203810,0.165127,-0.171441,0.297230,-0.054280,-0.012814,0.288070,-0.032690,0.248977,-0.132112,0.342373,-0.283569,-0.650882,-0.290329,0.175495,...,0.487582,-0.904762,-0.109912,0.383082,0.126106,-0.445692,-0.395702,-0.371305,-0.517765,-0.866404,-0.445692,-0.786441,-0.461694,0.475231,-0.948718,-0.189139,-0.317616,-0.704673,-0.621752,-0.586926,-0.579769,-0.555529,-0.733606,-0.621752,-0.916470,-0.561092,0.328573,-0.904762,-0.260239,0.132597,-0.154372,0.175643,0.513423,0.968562,-0.253992,-0.673257,0.323866,0.015887,2,WALKING_DOWNSTAIRS
1,0.239223,-0.031477,-0.105473,-0.895730,-0.910200,-0.944044,-0.907762,-0.913140,-0.943929,-0.908422,-0.534366,-0.782379,0.702528,0.636651,0.824235,-0.913950,-0.993854,-0.997755,-0.997822,-0.937716,-0.932928,-0.948821,0.064764,-0.434452,-0.326168,-0.293357,-0.016060,0.372616,-0.303446,-0.239328,-0.014961,0.432186,-0.310430,-0.017772,-0.097186,0.398385,-0.444456,-0.423243,-0.862583,0.209881,...,-0.898000,-1.000000,0.002040,-0.367677,-0.747877,-0.929916,-0.896370,-0.915577,-0.879163,-0.916564,-0.929916,-0.994477,-0.962295,-0.296197,-1.000000,-0.189233,0.341697,0.080852,-0.959017,-0.966154,-0.962918,-0.972842,-0.970014,-0.959017,-0.999121,-0.965801,-0.497993,-1.000000,0.074829,-0.362611,-0.746751,0.501495,0.112612,-0.118652,0.487422,0.385570,-0.631058,-0.335879,20,LAYING
2,0.272430,-0.017791,-0.105656,-0.986900,-0.987047,-0.990689,-0.987456,-0.986189,-0.990150,-0.932266,-0.570833,-0.810314,0.838099,0.689392,0.847325,-0.988626,-0.999853,-0.999918,-0.999824,-0.988616,-0.985671,-0.988558,-0.479287,-0.640648,-0.519156,-0.048398,-0.027917,0.078993,0.044116,0.278088,-0.139673,0.134070,0.152491,0.427384,-0.209650,0.199601,-0.210897,0.076618,-0.173813,-0.454288,...,-1.000000,-0.777778,0.503482,-0.797450,-0.971363,-0.991585,-0.992080,-0.992464,-0.990412,-0.994642,-0.991585,-0.999929,-0.993729,

In [ ]:
len(DF_test)

2947

In [ ]:
test_split = 0.3

random_clients = random.sample(range(1, 30), 5)
print('Randomly Selected 5 clients: ', random_clients)

C1_Train = DF[DF.subject == random_clients[0]].iloc[:int((1-test_split)*len(DF[DF.subject == random_clients[0]])), :]
C2_Train = DF[DF.subject == random_clients[1]].iloc[:int((1-test_split)*len(DF[DF.subject == random_clients[1]])), :]
C3_Train = DF[DF.subject == random_clients[2]].iloc[:int((1-test_split)*len(DF[DF.subject == random_clients[2]])), :]
C4_Train = DF[DF.subject == random_clients[3]].iloc[:int((1-test_split)*len(DF[DF.subject == random_clients[3]])), :]
C5_Train = DF[DF.subject == random_clients[4]].iloc[:int((1-test_split)*len(DF[DF.subject == random_clients[4]])), :]

C1_Test = DF[DF.subject == random_clients[0]].iloc[int((1-test_split)*len(DF[DF.subject == random_clients[0]])):, :]
C2_Test = DF[DF.subject == random_clients[1]].iloc[int((1-test_split)*len(DF[DF.subject == random_clients[1]])):, :]
C3_Test = DF[DF.subject == random_clients[2]].iloc[int((1-test_split)*len(DF[DF.subject == random_clients[2]])):, :]
C4_Test = DF[DF.subject == random_clients[3]].iloc[int((1-test_split)*len(DF[DF.subject == random_clients[3]])):, :]
C5_Test = DF[DF.subject == random_clients[4]].iloc[int((1-test_split)*len(DF[DF.subject == random_clients[4]])):, :]


Randomly Selected 5 clients:  [9, 10, 4, 21, 5]


In [ ]:
# Removing the data for our 5 clients from the training and testing sets
Train = pd.concat([DF, pd.concat([C1_Train, C2_Train, C3_Train, C4_Train, C5_Train, C1_Test, C2_Test, C3_Test, C4_Test, C5_Test])]).drop_duplicates(keep=False).iloc[:int((1-test_split)*len(DF)), :]
Test = pd.concat([DF, pd.concat([C1_Train, C2_Train, C3_Train, C4_Train, C5_Train, C1_Test, C2_Test, C3_Test, C4_Test, C5_Test])]).drop_duplicates(keep=False).iloc[int((1-test_split)*len(DF)):, :]


In [ ]:
# ----------------------------- #
# ---------- SETTINGS ----------#
# ----------------------------- #

NUM_Clients = 5 # number of clients contributing per training round

# ML
Cluster_Size = 100 # max client dataset size for training
Batch_Size = 10
NUM_Epoch = 3
verbose = 1

# Krum
krum_f = 0.25 # percentage of byzantine nodes

# Differential Privacy
Gaussian_Noise = False
Gaussian_Noise_Std_Dev = 0.25

Gradient_Clipping = False
Clip_Norm = 0.60

Gradient_Pruning = False
initial_sparsity = 0.00
final_sparsity = 0.50

# ---------------------------- #
# ----------------------------- #
# ----------------------------- #

In [ ]:
def preprocess(df):
  y_train = df.Activity
  X_train = df.drop(['subject', 'Activity'], axis=1)
  print('Training data size : ', X_train.shape)
  labelencoder = LabelEncoder()
  y_train = labelencoder.fit_transform(y_train)
  y_train = to_categorical(y_train)
  sc = StandardScaler()
  X_train = sc.fit_transform(X_train)
  print('X_Train Shape: ', X_train.shape)
  print('y_Train Shape: ', y_train.shape)
  return X_train, y_train

In [ ]:
X_test, y_test = preprocess(Test)

Training data size :  (1481, 561)
X_Train Shape:  (1481, 561)
y_Train Shape:  (1481, 6)


In [ ]:
def train(name, X_train, y_train, globalId):

  n_timesteps, n_features, n_outputs = X_train.shape[0], X_train.shape[1], y_train.shape[0]

  model = Sequential()

  model.add(Dense(100, activation='relu', input_dim=n_features))
  if Gaussian_Noise == True:
        model.add(GaussianNoise(Gaussian_Noise_Std_Dev))
  model.add(Dropout(0.1))
  model.add(Dense(100, activation='relu'))
  model.add(Dropout(0.1))
  model.add(Dense(6, activation='softmax'))

  if Gradient_Pruning == True:
        end_step = np.ceil(n_timesteps / Batch_Size).astype(np.int32) * NUM_Epoch

        # Define model for pruning.
        pruning_params = {
              'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=initial_sparsity,
                                                                       final_sparsity=final_sparsity,
                                                                       begin_step=0,
                                                                       end_step=end_step)
        }

        logdir = tempfile.mkdtemp()

        callbacks = [
          tfmot.sparsity.keras.UpdatePruningStep(),
          tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
        ]

        model = prune_low_magnitude(model, **pruning_params)

  if globalId != 1:
        model.load_weights("./weights/global"+str(globalId)+".h5")

  if Gradient_Clipping == True:
    opt = keras.optimizers.Adam(clipnorm=Clip_Norm)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
  else: 
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
      
  if Gradient_Pruning == True:
    history = model.fit(X_train, y_train, epochs=NUM_Epoch, batch_size=Batch_Size, verbose=1, callbacks=callbacks)
  else:
    history = model.fit(X_train, y_train, epochs=NUM_Epoch, batch_size=Batch_Size, verbose=1)
  
  #Saving Model
  model.save("./weights/"+str(name)+".h5")
  return n_timesteps, model

In [ ]:
def euclidean(m, n):
    # Finds eucledian distance between two ML models m & n
    distance = []
    for i in range(len(m)):
        distance.append(euc(m[i].reshape(-1,1), n[i].reshape(-1,1)))
    distance = sum(distance)/len(m)
    return distance

def saveModel(weight, n):
    
    num_classes=len(np.unique(y_test))

    model = Sequential()

    model.add(Dense(100, activation='relu', input_dim=561))
    if Gaussian_Noise == True:
          model.add(GaussianNoise(Gaussian_Noise_Std_Dev))
    model.add(Dropout(0.1))
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(6, activation='softmax'))

    if Gradient_Pruning == True:
          model = prune_low_magnitude(model)

    model.set_weights(weight)

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    scores = model.evaluate(X_test, y_test)
    print("Loss: ", scores[0])        #Loss
    print("Accuracy: ", scores[1])    #Accuracy

    #Saving Model
    fpath = "./weights/global"+str(n)+".h5"
    model.save(fpath)
    return scores[0], scores[1]

def getDataLen(trainingDict):
    n = 0
    for w in trainingDict:
        n += trainingDict[w]
    print('Total number of data points after this round: ', n)
    return n

def assignWeights(trainingDf, trainingDict):
    n = getDataLen(trainingDict)
    trainingDf['Weightage'] = trainingDf['DataSize'].apply(lambda x: x/n)
    return trainingDf, n
    
def scale(weight, scaler):
    scaledWeights = []
    for i in range(len(weight)):
        scaledWeights.append(scaler * weight[i])
    return scaledWeights

def getScaledWeight(d, scaler):
    model = Sequential()
    model.add(Dense(100, activation='relu', input_dim=561))
    if Gaussian_Noise == True:
          model.add(GaussianNoise(Gaussian_Noise_Std_Dev))
    model.add(Dropout(0.1))
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(6, activation='softmax'))

    if Gradient_Pruning == True:
        model = prune_low_magnitude(model)
    fpath = "./weights/"+d+".h5"
    model.load_weights(fpath)
    weight = model.get_weights()
    return scale(weight, scaler)

def getWeight(d):
    model = Sequential()
    model.add(Dense(100, activation='relu', input_dim=561))
    if Gaussian_Noise == True:
          model.add(GaussianNoise(Gaussian_Noise_Std_Dev))
    model.add(Dropout(0.1))
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(6, activation='softmax'))

    if Gradient_Pruning == True:
        model = prune_low_magnitude(model)
    fpath = "./weights/"+d+".h5"
    model.load_weights(fpath)
    weight = model.get_weights()
    return weight

def avgWeights(scaledWeights):
    avg = list()
    for weight_list_tuple in zip(*scaledWeights):
        layer_mean = tf.math.reduce_sum(weight_list_tuple, axis=0)
        avg.append(layer_mean)
    return avg

def FedAvg(trainingDict):
    trainingDf = pd.DataFrame.from_dict(trainingDict, orient='index', columns=['DataSize']) 
    models = list(trainingDict.keys())
    scaledWeights = []
    trainingDf, dataLen = assignWeights(trainingDf, trainingDict)
    for m in models:
        scaledWeights.append(getScaledWeight(m, trainingDf.loc[m]['Weightage']))
    fedAvgWeight = avgWeights(scaledWeights)
    return fedAvgWeight, dataLen

def MK(trainingDict, b):
    models = list(trainingDict.keys())
    trainingDf = pd.DataFrame.from_dict(trainingDict, orient='index', columns=['DataSize'])
    l_weights = []
    g_weight = {}
    for m in models:
        if 'global' in m:
            g_weight['name'] = m
            g_weight['weight'] = getWeight(m)
        else:
            l_weights.append({
                'name': m,
                'weight': getWeight(m)
            })
    scores = {}
    for m in l_weights:
        scores[m['name']] = euclidean(m['weight'], g_weight['weight'])
    sortedScores = {k: v for k, v in sorted(scores.items(), key=lambda item: item[1])}

    b = int(len(scores)*b)
    
    selected = []
    for i in range(b):
        selected.append((sortedScores.popitem())[0])

    newDict = {}
    for i in trainingDict.keys():
        if (((i not in selected) and ('global' not in i))):
            newDict[i] = trainingDict[i]

    print('Selections: ', newDict)
    NewGlobal, dataLen = FedAvg(newDict)
    return NewGlobal, dataLen


In [ ]:
curr_local = 0
curr_global = 0
def fed():
  global curr_local
  global curr_global

  local = {}
  loss_array = []
  acc_array = []
  for i in range(0, len(Train), Cluster_Size):

      if int(curr_global) == 0:
        curr_global += 1
        print('Current Global: ', curr_global)
        name = 'global' + str(curr_global)
        X_train, y_train = preprocess(Train[i:i+Cluster_Size])
        l, m = train(name, X_train, y_train, curr_global)
        local[name] = l

      else:
        print('Current Local: ', curr_local)
        name = str('local'+str(curr_local))
        curr_local += 1
        X_train, y_train = preprocess(Train[i:i+Cluster_Size])
        if X_train.shape[0]<=99:
          continue
        l, m = train(name, X_train, y_train, curr_global)
        local[name] = l

        if (int(curr_local)%NUM_Clients == 0) and (curr_local != 0):
          curr_global += 1
          print('Current Global: ', curr_global)
          name = 'global' + str(curr_global)
          m, l = MK(local, krum_f)
          loss, acc = saveModel(m, curr_global)
          loss_array.append(loss)
          acc_array.append(acc)
          local = {}
          local[name] = l
  return acc_array, loss_array

In [ ]:
fed()

Current Global:  1
Training data size :  (100, 561)
X_Train Shape:  (100, 561)
y_Train Shape:  (100, 6)
Epoch 1/3
10/10 [==============================] - 1s 2ms/step - loss: 1.7357 - accuracy: 0.2964
Epoch 2/3
10/10 [==============================] - 0s 2ms/step - loss: 0.7289 - accuracy: 0.7625
Epoch 3/3
10/10 [==============================] - 0s 2ms/step - loss: 0.5226 - accuracy: 0.7743
Current Local:  0
Training data size :  (100, 561)
X_Train Shape:  (100, 561)
y_Train Shape:  (100, 6)
Epoch 1/3
10/10 [==============================] - 0s 2ms/step - loss: 1.6581 - accuracy: 0.3119
Epoch 2/3
10/10 [==============================] - 0s 2ms/step - loss: 0.7491 - accuracy: 0.6947
Epoch 3/3
10/10 [==============================] - 0s 2ms/step - loss: 0.5237 - accuracy: 0.8028
Current Local:  1
Training data size :  (100, 561)
X_Train Shape:  (100, 561)
y_Train Shape:  (100, 6)
Epoch 1/3
10/10 [==============================] - 0s 2ms/step - loss: 2.0213 - accuracy: 0.2788
Epoch 2/3
1

([0.5037137269973755,
  0.8203916549682617,
  0.8332207798957825,
  0.8838622570037842,
  0.9068197011947632,
  0.922349750995636,
  0.9358541369438171,
  0.945307195186615,
  0.9291019439697266,
  0.9513841867446899,
  0.9561107158660889,
  0.948008120059967,
  0.9574611783027649,
  0.9534098505973816],
 [1.6105306148529053,
  0.6908660531044006,
  0.44823262095451355,
  0.31242284178733826,
  0.2495577335357666,
  0.19763822853565216,
  0.17301638424396515,
  0.14828135073184967,
  0.16707688570022583,
  0.12153568118810654,
  0.12394406646490097,
  0.12955030798912048,
  0.10837368667125702,
  0.12818479537963867])

In [ ]:
def getNoTL(X_test, y_test, curr_global):
    num_classes=len(np.unique(y_test))

    inner_model = Sequential(
    [
        Dense(100, activation='relu', input_dim=561)
    ]
)

    model = Sequential()

    model.add(inner_model)

    if Gaussian_Noise == True:
          model.add(GaussianNoise(Gaussian_Noise_Std_Dev))
    model.add(Dropout(0.1))
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(6, activation='softmax'))


    if Gradient_Pruning == True:
          end_step = np.ceil(16500 / Batch_Size).astype(np.int32) * NUM_Epoch

          # Define model for pruning.
          pruning_params = {
                'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=initial_sparsity,
                                                                        final_sparsity=final_sparsity,
                                                                        begin_step=0,
                                                                        end_step=end_step)
          }

          logdir = tempfile.mkdtemp()

          callbacks = [
            tfmot.sparsity.keras.UpdatePruningStep(),
            tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
          ]

          model = prune_low_magnitude(model, **pruning_params)

    model.load_weights("./weights/global"+str(curr_global)+".h5")

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
      
    scores = model.evaluate(X_test, y_test)
    print("NoTL Loss: ", scores[0])        #Loss
    print("NoTL Accuracy: ", scores[1])    #Accuracy

In [ ]:
def getGlobalTest(testdf, curr_global):

    X_test, y_test = preprocess(testdf)

    num_classes=len(np.unique(y_test))

    inner_model = Sequential(
    [
        Dense(100, activation='relu', input_dim=561)
    ]
)

    model = Sequential()

    model.add(inner_model)

    if Gaussian_Noise == True:
          model.add(GaussianNoise(Gaussian_Noise_Std_Dev))
    model.add(Dropout(0.1))
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(6, activation='softmax'))


    if Gradient_Pruning == True:
          end_step = np.ceil(16500 / Batch_Size).astype(np.int32) * NUM_Epoch

          # Define model for pruning.
          pruning_params = {
                'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=initial_sparsity,
                                                                        final_sparsity=final_sparsity,
                                                                        begin_step=0,
                                                                        end_step=end_step)
          }

          logdir = tempfile.mkdtemp()

          callbacks = [
            tfmot.sparsity.keras.UpdatePruningStep(),
            tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
          ]

          model = prune_low_magnitude(model, **pruning_params)

    model.load_weights("./weights/global"+str(curr_global)+".h5")

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
      
    scores = model.evaluate(X_test, y_test)
    print("GlobalTest Loss: ", scores[0])        #Loss
    print("GlobalTestA Accuracy: ", scores[1])    #Accuracy

In [ ]:
def TransferLearn(name, traindf, testdf):
  global curr_global
  X_train, y_train = preprocess(traindf)
  X_test, y_test = preprocess(testdf)

  n_timesteps, n_features, n_outputs = X_train.shape[0], X_train.shape[1], y_train.shape[0]

  inner_model = Sequential(
    [
        Dense(100, activation='relu', input_dim=561)
    ]
)

  model = Sequential()

  model.add(inner_model)

  if Gaussian_Noise == True:
        model.add(GaussianNoise(Gaussian_Noise_Std_Dev))
  model.add(Dropout(0.1))
  model.add(Dense(100, activation='relu'))
  model.add(Dropout(0.1))
  model.add(Dense(6, activation='softmax'))


  if Gradient_Pruning == True:
        end_step = np.ceil(n_timesteps / Batch_Size).astype(np.int32) * NUM_Epoch

        # Define model for pruning.
        pruning_params = {
              'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=initial_sparsity,
                                                                       final_sparsity=final_sparsity,
                                                                       begin_step=0,
                                                                       end_step=end_step)
        }

        logdir = tempfile.mkdtemp()

        callbacks = [
          tfmot.sparsity.keras.UpdatePruningStep(),
          tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
        ]

        model = prune_low_magnitude(model, **pruning_params)

  model.load_weights("./weights/global"+str(curr_global)+".h5")

  if Gradient_Clipping == True:
    opt = keras.optimizers.Adam(clipnorm=Clip_Norm)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
  else: 
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

  for layer in inner_model.layers:#freezing layers to retain the weights 
	  layer.trainable = False
      
  if Gradient_Pruning == True:
    history = model.fit(X_train, y_train, epochs=5, batch_size=Batch_Size, verbose=1, callbacks=callbacks)
  else:
    history = model.fit(X_train, y_train, epochs=5, batch_size=Batch_Size, verbose=1)

  history2 = model.evaluate(X_test, y_test, batch_size=Batch_Size, verbose=1)

  test_accuracy = history2[1]
  print(f"The test accuracy after Transfer Learning is {test_accuracy}")

  getNoTL(X_test, y_test, curr_global)
  getGlobalTest(Test, curr_global)

  X_test_G, y_test_G = preprocess(Test)

  s3 = model.evaluate(X_test_G, y_test_G, batch_size=Batch_Size, verbose=1)

  print("After TL Global Test Loss: ", s3[0])        #Loss
  print("After TL Global Test Accuracy: ", s3[1])    #Accuracy


  
  #Saving Model
  model.save("./weights/"+str(name)+".h5")
  return n_timesteps, model

In [ ]:
TransferLearn('C1', C1_Train, C1_Test)

Training data size :  (201, 561)
X_Train Shape:  (201, 561)
y_Train Shape:  (201, 6)
Training data size :  (87, 561)
X_Train Shape:  (87, 561)
y_Train Shape:  (87, 6)
Epoch 1/5
21/21 [==============================] - 0s 3ms/step - loss: 0.3991 - accuracy: 0.8770
Epoch 2/5
21/21 [==============================] - 0s 2ms/step - loss: 0.2766 - accuracy: 0.8787
Epoch 3/5
21/21 [==============================] - 0s 3ms/step - loss: 0.1832 - accuracy: 0.9073
Epoch 4/5
21/21 [==============================] - 0s 3ms/step - loss: 0.0980 - accuracy: 0.9486
Epoch 5/5
9/9 [==============================] - 0s 2ms/step - loss: 0.1340 - accuracy: 0.9425
The test accuracy after Transfer Learning is 0.9425287246704102
3/3 [==============================] - 0s 4ms/step - loss: 0.6854 - accuracy: 0.8026
NoTL Loss:  0.5927228331565857
NoTL Accuracy:  0.8160919547080994
Training data size :  (1481, 561)
X_Train Shape:  (1481, 561)
y_Train Shape:  (1481, 6)
47/47 [==============================] - 0s 2ms

(201, <tensorflow.python.keras.engine.sequential.Sequential at 0x7f28de31f5d0>)

In [ ]:
TransferLearn('C2', C2_Train, C2_Test)

Training data size :  (205, 561)
X_Train Shape:  (205, 561)
y_Train Shape:  (205, 6)
Training data size :  (89, 561)
X_Train Shape:  (89, 561)
y_Train Shape:  (89, 6)
Epoch 1/5
21/21 [==============================] - 1s 2ms/step - loss: 0.2849 - accuracy: 0.9231
Epoch 2/5
21/21 [==============================] - 0s 2ms/step - loss: 0.0686 - accuracy: 0.9857
Epoch 3/5
21/21 [==============================] - 0s 2ms/step - loss: 0.0404 - accuracy: 0.9905
Epoch 4/5
21/21 [==============================] - 0s 2ms/step - loss: 0.0119 - accuracy: 0.9951
Epoch 5/5
9/9 [==============================] - 0s 2ms/step - loss: 0.0205 - accuracy: 1.0000
The test accuracy after Transfer Learning is 1.0
3/3 [==============================] - 0s 4ms/step - loss: 0.3222 - accuracy: 0.8571
NoTL Loss:  0.3989662528038025
NoTL Accuracy:  0.8314606547355652
Training data size :  (1481, 561)
X_Train Shape:  (1481, 561)
y_Train Shape:  (1481, 6)
47/47 [==============================] - 0s 2ms/step - loss: 0

(205, <tensorflow.python.keras.engine.sequential.Sequential at 0x7f28e5d96510>)

In [ ]:
TransferLearn('C3', C3_Train, C3_Test)

Training data size :  (221, 561)
X_Train Shape:  (221, 561)
y_Train Shape:  (221, 6)
Training data size :  (96, 561)
X_Train Shape:  (96, 561)
y_Train Shape:  (96, 6)
Epoch 1/5
23/23 [==============================] - 0s 2ms/step - loss: 0.2104 - accuracy: 0.9549
Epoch 2/5
23/23 [==============================] - 0s 2ms/step - loss: 0.0420 - accuracy: 0.9683
Epoch 3/5
23/23 [==============================] - 0s 3ms/step - loss: 0.0510 - accuracy: 0.9806
Epoch 4/5
23/23 [==============================] - 0s 3ms/step - loss: 0.0216 - accuracy: 1.0000
Epoch 5/5
10/10 [==============================] - 0s 2ms/step - loss: 0.0495 - accuracy: 0.9792
The test accuracy after Transfer Learning is 0.9791666865348816
3/3 [==============================] - 0s 4ms/step - loss: 0.1487 - accuracy: 0.9466
NoTL Loss:  0.1253236085176468
NoTL Accuracy:  0.9479166865348816
Training data size :  (1481, 561)
X_Train Shape:  (1481, 561)
y_Train Shape:  (1481, 6)
47/47 [==============================] - 0s 2

(221, <tensorflow.python.keras.engine.sequential.Sequential at 0x7f28f60dee90>)

In [ ]:
TransferLearn('C4', C4_Train, C4_Test)

Training data size :  (285, 561)
X_Train Shape:  (285, 561)
y_Train Shape:  (285, 6)
Training data size :  (123, 561)
X_Train Shape:  (123, 561)
y_Train Shape:  (123, 6)
Epoch 1/5
29/29 [==============================] - 0s 2ms/step - loss: 0.2669 - accuracy: 0.9005
Epoch 2/5
29/29 [==============================] - 0s 2ms/step - loss: 0.0573 - accuracy: 0.9765
Epoch 3/5
29/29 [==============================] - 0s 2ms/step - loss: 0.0300 - accuracy: 0.9960
Epoch 4/5
29/29 [==============================] - 0s 2ms/step - loss: 0.0125 - accuracy: 0.9976
Epoch 5/5
13/13 [==============================] - 0s 2ms/step - loss: 0.0170 - accuracy: 1.0000
The test accuracy after Transfer Learning is 1.0
4/4 [==============================] - 0s 3ms/step - loss: 0.2169 - accuracy: 0.9280
NoTL Loss:  0.23926423490047455
NoTL Accuracy:  0.9268292784690857
Training data size :  (1481, 561)
X_Train Shape:  (1481, 561)
y_Train Shape:  (1481, 6)
47/47 [==============================] - 0s 2ms/step - l

(285, <tensorflow.python.keras.engine.sequential.Sequential at 0x7f28dfd3b650>)

In [ ]:
TransferLearn('C5', C5_Train, C5_Test)

Training data size :  (211, 561)
X_Train Shape:  (211, 561)
y_Train Shape:  (211, 6)
Training data size :  (91, 561)
X_Train Shape:  (91, 561)
y_Train Shape:  (91, 6)
Epoch 1/5
22/22 [==============================] - 0s 2ms/step - loss: 0.3440 - accuracy: 0.8962
Epoch 2/5
22/22 [==============================] - 0s 3ms/step - loss: 0.1756 - accuracy: 0.9338
Epoch 3/5
22/22 [==============================] - 0s 3ms/step - loss: 0.1159 - accuracy: 0.9522
Epoch 4/5
22/22 [==============================] - 0s 3ms/step - loss: 0.0765 - accuracy: 0.9689
Epoch 5/5
10/10 [==============================] - 0s 2ms/step - loss: 0.0728 - accuracy: 0.9670
The test accuracy after Transfer Learning is 0.9670329689979553
3/3 [==============================] - 0s 4ms/step - loss: 0.3124 - accuracy: 0.9021
NoTL Loss:  0.3909125328063965
NoTL Accuracy:  0.8901098966598511
Training data size :  (1481, 561)
X_Train Shape:  (1481, 561)
y_Train Shape:  (1481, 6)
47/47 [==============================] - 0s 2

(211, <tensorflow.python.keras.engine.sequential.Sequential at 0x7f28e5d8e050>)

In [ ]:
!ls

 hapt-fl.png   mnist	      rt_reviews.csv   train.csv	  weights
 malaria       mnist-fl.png   test.csv	      'UCI HAR Dataset'
